In [1]:
%cd /Users/maimai/Desktop/project/May


/Users/maimai/Desktop/project/May


In [2]:
import builtins
builtins.uclresearch_topic = 'GIVENCHY' # 226984 entires
# builtins.uclresearch_topic = 'HAWKING' # 4828104 entries
# builtins.uclresearch_topic = 'NYC'
# builtins.uclresearch_topic = 'FLORIDA'
from configuration import config

('Reading config file from location: '
 '/Users/maimai/Desktop/project/May/configuration/env.properties')
{'calculate': {'analysis': True,
               'friends': True,
               'network': True,
               'uniquetweets': True,
               'uniqueusers': True},
 'data': {'dates': ['2018-03-11', '2018-03-12', '2018-03-13'],
          'eventname': "Givenchy's Death",
          'phrases': ['givenchy%20death', 'givenchy%20passed%20away'],
          'starttime': 'Mar 12 08:20:00 -0500 2018'},
 'path': {'cwd': '/Users/maimai/Desktop/project/May/data/givenchy',
          'ml': '/Users/maimai/Desktop/project/May/data/givenchy/pickle',
          'networkx': {'all': '/Users/maimai/Desktop/project/May/data/givenchy/pickle/networkx_all.dat',
                       'friends': '/Users/maimai/Desktop/project/May/data/givenchy/pickle/networkx_friends.dat',
                       'potential': '/Users/maimai/Desktop/project/May/data/givenchy/pickle/networkx_potential.dat'},
          'new

In [3]:
import networkx as nx
from tqdm import tqdm
import pandas as pd
import numpy as np
import math
import multiprocessing
from multiprocessing import Pool
from IPython.display import display

In [4]:
unique_users = config.load_users_dataframe()
network_friends = config.load_networkx_friends()

Loading data file from path /Users/maimai/Desktop/project/May/data/givenchy/pickle/users.dat
'Loaded 5011 entires'
Loading data file from path /Users/maimai/Desktop/project/May/data/givenchy/pickle/networkx_friends.dat
'Loaded 5011 entires'


In [5]:
unique_users

,user,user_id,time_lapsed,followers_count,friends_count,user_created_days,user_statuses_count,user_listed_count,user_favourites_count,normalized_user_statuses_count,normalized_user_followers_count,normalized_user_favourites_count,normalized_user_listed_count,normalized_user_friends_count,mention_and_reply,source_candidates,source_index,seed_index,generation,time_since_seed
0,davelackie,100766356,0.00,143119,4643,2993,181393,555,11938,60.605747,47.817908,3.988640,0.185433,1.551286,[],"[3, 5, 6, 7, 10, 11, 14, 15, 19, 20, 24, 74, 8...",None,0,0,0
1,alexanderskhan,3186545203,0.15,753,3428,1042,12853,8,34186,12.334933,0.722649,32.808061,0.007678,3.289827,[0],[0],0,0,1,0.15
2,consiglierela,4134992843,0.23,2161,396,856,46934,36,78797,54.829439,2.524533,92.052570,0.042056,0.462617,[0],"[0, 12, 21, 32, 48, 748, 819, 972, 2358, 2778,...",0,0,1,0.23
3,ImpactPrincess,1955472014,0.63,242,642,1613,23179,55,28995,14.370118,0.150031,17.975821,0.034098,0.398016,[0],"[0, 115, 577]",0,0,1,0.63
4,Damek0Masca,406301175,1.13,116,239,2318,2407,12,1058,1.038395,0.050043,0.456428,0.005177,0.103106,[],[],None,4,0,0
5,panagiota_karag,2938214163,1.18,250,493,1176,18995,20,13390,16.152211,0.212585,11.386054,0.017007,0.419218,[0],"[0, 20, 577, 1012, 1591, 1656, 1807, 1953, 320...",0,0,1,1.18
6,Gwen_UsBeauty,89805327,1.32,39697,1924,3041,34644,528,20885,11.392305,13.053930,6.867807,0.173627,0.632687,[0],"[0, 21, 748, 1912, 1929]",0,0,1,1.32
7,mascott04,342931336,1.83,672,353,2421,15651,1,54298,6.464684,0.277571,22.427922,0.000413,0.145808,[0],[0],0,0,1,1.83
8,FazeyPie,27220084,2.15,275,440,3272,25944,21,13821,7.929095,0.084046,4.224022,0.006418,0.134474,[0],[0],0,0,1,2.15
9,RissaRunsThis,97360017,2.42,1911,506,3008,175675,80,3810,58.402593,0.635306,1.266622,0.026596,0.168218,[0],[0],0,0,1,2.42


# Settings

In [6]:
# intervals = [60, 30, 15, 7]
intervals = [30]

# Helper function for mean value calculation
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

# Pre-Calculate features

In [7]:
%%time
# Calculating in and out degrees
nodeInDegreeDict = network_friends.in_degree()
nodeOutDegreeDict = network_friends.out_degree()

CPU times: user 32 µs, sys: 8 µs, total: 40 µs
Wall time: 45.1 µs


In [8]:
%%time
convert_dictionary_to_sorted_list = lambda x: [x[a] for a in sorted(x)]

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 10 µs


Assortativity [https://networkx.github.io/documentation/networkx-1.9.1/reference/algorithms.assortativity.html]

In [9]:
%%time
average_neighbor_degree = convert_dictionary_to_sorted_list(nx.average_neighbor_degree(network_friends))

CPU times: user 66.2 ms, sys: 2.08 ms, total: 68.3 ms
Wall time: 67.3 ms


Centrality [https://networkx.github.io/documentation/networkx-1.9.1/reference/algorithms.centrality.html]

In [10]:
%%time
degree_centrality = convert_dictionary_to_sorted_list(nx.degree_centrality(network_friends))

CPU times: user 5.11 ms, sys: 84 µs, total: 5.19 ms
Wall time: 5.14 ms


In [11]:

%%time
in_degree_centrality = convert_dictionary_to_sorted_list(nx.in_degree_centrality(network_friends))

CPU times: user 3.84 ms, sys: 100 µs, total: 3.94 ms
Wall time: 3.87 ms


In [12]:
%%time
out_degree_centrality = convert_dictionary_to_sorted_list(nx.out_degree_centrality(network_friends))

CPU times: user 3.68 ms, sys: 24 µs, total: 3.7 ms
Wall time: 3.73 ms


In [13]:
# Too slow
# %%time
# closeness_centrality = convert_dictionary_to_sorted_list(nx.closeness_centrality(network_friends))

In [14]:
# Too slow
# %%time
# betweenness_centrality = convert_dictionary_to_sorted_list(nx.betweenness_centrality(network_friends))

In [15]:
%%time
eigenvector_centrality = convert_dictionary_to_sorted_list(nx.eigenvector_centrality(network_friends))

CPU times: user 1.83 s, sys: 10.6 ms, total: 1.84 s
Wall time: 1.85 s


# Feature Extraction

In [18]:
def process_data(interval, start_index, end_index):
    features = {
        'label': [],
        'user_id': [],
        # UsM: User metadata
        'UsM_deltaDays': [],
        'UsM_statusesCount': [],
        'UsM_followersCount': [],
        'UsM_favouritesCount': [],
        'UsM_friendsCount': [],
        'UsM_listedCount': [],
        'UsM_normalizedUserStatusesCount': [],
        'UsM_normalizedUserFollowersCount': [],
        'UsM_normalizedUserFavouritesCount': [],
        'UsM_normalizedUserListedCount': [],
        'UsM_normalizedUserFriendsCount': [],                 
        'UsM_deltaDays0': [],
        'UsM_statusesCount0': [],
        'UsM_followersCount0': [],
        'UsM_favouritesCount0': [],
        'UsM_friendsCount0': [],
        'UsM_listedCount0': [],
        'UsM_normalizedUserStatusesCount0': [],
        'UsM_normalizedUserFollowersCount0': [],
        'UsM_normalizedUserFavouritesCount0': [],
        'UsM_normalizedUserListedCount0': [],
        'UsM_normalizedUserFriendsCount0': [],
        'UsM_deltaDays-1': [],
        'UsM_statusesCount-1': [],
        'UsM_followersCount-1': [],
        'UsM_favouritesCount-1': [],
#         'UsM_friendsCount-1': [],
        'UsM_listedCount-1': [],
        'UsM_normalizedUserStatusesCount-1': [],
        'UsM_normalizedUserFollowersCount-1': [],
        'UsM_normalizedUserFavouritesCount-1': [],
        'UsM_normalizedUserListedCount-1': [],
        'UsM_normalizedUserFriendsCount-1': [],
        # TwM: Tweet metadata
        'TwM_t0': [],
        'TwM_t-1': [],
        'TwM_tCurrent': [],
        # Nw: Network
        'Nw_nNodes': [],
        'Nw_averageNeighborDegree': [],
        'Nw_degreeCentrality': [],
        'Nw_inDegreeCentrality': [],
        'Nw_outDegreeCentrality': [],
        'Nw_eigenvectorCentrality': [],
        'Nw_averageNeighborDegree0': [],
        'Nw_degreeCentrality0': [],
        'Nw_inDegreeCentrality0': [],
        'Nw_outDegreeCentrality0': [],
        'Nw_eigenvectorCentrality0': [],
        'Nw_averageNeighborDegree-1': [],
        'Nw_degreeCentrality-1': [],
        'Nw_inDegreeCentrality-1': [],
        'Nw_outDegreeCentrality-1': [],
        'Nw_eigenvectorCentrality-1': [],
        # Stat: Statistical
        'Stat_average_kOut': [],
        'Stat_average_t': [],
        'Stat_average_deltaDays': [],
        'Stat_average_statusesCount': [],
        'Stat_average_followersCount': [],
        'Stat_average_favouritesCount': [],
        'Stat_average_friendsCount': [],
        'Stat_average_listedCount': [],
        'Stat_average_normalizedUserStatusesCount': [],
        'Stat_average_normalizedUserFollowersCount': [],
        'Stat_average_normalizedUserFavouritesCount': [],
        'Stat_average_normalizedUserListedCount': [],
        'Stat_average_normalizedUserFriendsCount': [],                
        'Stat_max_kOut': [],
        'Stat_min_kOut': []
    }

    with tqdm(total=len(list(unique_users[start_index: end_index].iterrows()))) as pbar: 
        for index, user_row in unique_users[start_index: end_index].iterrows():
            if user_row['source_index'] is not None:          
                source_candidates = user_row['source_candidates']
                source_first = source_candidates[0]
                source_first_row = unique_users.iloc[source_first]
                source_first_time_lapsed = source_first_row.time_lapsed
                start_bar = int(source_first_time_lapsed / interval) + 1                
                bars = list(np.arange(start_bar * interval, 24 * 60, interval))
                number_of_bars = len(bars)
                
                for current_time in bars:
                    # all sources up to the current time
                    sources = [x for x in source_candidates if unique_users.iloc[x].time_lapsed <= current_time]
                    sources_dataframe = unique_users.iloc[sources]
                    
                    averageNeighborDegreeList = list(average_neighbor_degree[i] for i in sources)
                    degreeCentralityList = list(degree_centrality[i] for i in sources)
                    inDegreeCentralityList = list(in_degree_centrality[i] for i in sources)
                    outDegreeCentralityList = list(out_degree_centrality[i] for i in sources)
                    eigenvectorCentralityList = list(eigenvector_centrality[i] for i in sources)
                    
                    degreeList = [nodeOutDegreeDict[x] for x in sources]
                    timeList = [current_time - unique_users.iloc[x].time_lapsed for x in sources]
                    
                    first_source_index = sources[0]
                    first_source_row = unique_users.iloc[first_source_index]
                    last_source_index = sources[-1]
                    last_source_row = unique_users.iloc[last_source_index]

                    #Extraction
                    features['label'].append(int(current_time >= user_row['time_lapsed']))
                    features['user_id'].append(user_row['user_id'])
                    # UsM: User metadata                    
                    features['UsM_deltaDays'].append(user_row['user_created_days'])
                    features['UsM_statusesCount'].append(user_row['user_statuses_count'])
                    features['UsM_followersCount'].append(user_row['followers_count'])
                    features['UsM_favouritesCount'].append(user_row['user_favourites_count'])
                    features['UsM_friendsCount'].append(user_row['friends_count'])
                    features['UsM_listedCount'].append(user_row['user_listed_count'])
                    features['UsM_normalizedUserStatusesCount'].append(user_row['normalized_user_statuses_count'])
                    features['UsM_normalizedUserFollowersCount'].append(user_row['normalized_user_followers_count'])
                    features['UsM_normalizedUserFavouritesCount'].append(user_row['normalized_user_favourites_count'])
                    features['UsM_normalizedUserListedCount'].append(user_row['normalized_user_listed_count'])
                    features['UsM_normalizedUserFriendsCount'].append(user_row['normalized_user_friends_count'])                 
                    features['UsM_deltaDays0'].append(source_first_row.user_created_days)
                    features['UsM_statusesCount0'].append(source_first_row.user_statuses_count)
                    features['UsM_followersCount0'].append(source_first_row.followers_count)
                    features['UsM_favouritesCount0'].append(source_first_row.user_favourites_count)
                    features['UsM_friendsCount0'].append(source_first_row.friends_count)
                    features['UsM_listedCount0'].append(source_first_row.user_listed_count)
                    features['UsM_normalizedUserStatusesCount0'].append(source_first_row.normalized_user_statuses_count)
                    features['UsM_normalizedUserFollowersCount0'].append(source_first_row.normalized_user_followers_count)
                    features['UsM_normalizedUserFavouritesCount0'].append(source_first_row.normalized_user_favourites_count)
                    features['UsM_normalizedUserListedCount0'].append(source_first_row.normalized_user_listed_count)
                    features['UsM_normalizedUserFriendsCount0'].append(source_first_row.normalized_user_friends_count)
                    features['UsM_deltaDays-1'].append(last_source_row.user_created_days)
                    features['UsM_statusesCount-1'].append(last_source_row.user_statuses_count)
                    features['UsM_followersCount-1'].append(last_source_row.followers_count)
                    features['UsM_favouritesCount-1'].append(last_source_row.user_favourites_count)
#                     features['UsM_friendsCount-1'].append(last_source_row.friends_count)
                    features['UsM_listedCount-1'].append(last_source_row.user_listed_count)
                    features['UsM_normalizedUserStatusesCount-1'].append(last_source_row.normalized_user_statuses_count)
                    features['UsM_normalizedUserFollowersCount-1'].append(last_source_row.normalized_user_followers_count)
                    features['UsM_normalizedUserFavouritesCount-1'].append(last_source_row.normalized_user_favourites_count)
                    features['UsM_normalizedUserListedCount-1'].append(last_source_row.normalized_user_listed_count)
                    features['UsM_normalizedUserFriendsCount-1'].append(last_source_row.normalized_user_friends_count) 
                    # TwM: Tweet metadata
                    features['TwM_t0'].append(round(timeList[0], 1))
                    features['TwM_t-1'].append(round(timeList[-1], 1))
                    features['TwM_tCurrent'].append(current_time)
                    # Nw: Network
                    features['Nw_nNodes'].append(len(sources))
                    features['Nw_averageNeighborDegree'].append(average_neighbor_degree[index])
                    features['Nw_degreeCentrality'].append(degree_centrality[index])
                    features['Nw_inDegreeCentrality'].append(in_degree_centrality[index])
                    features['Nw_outDegreeCentrality'].append(out_degree_centrality[index])
                    features['Nw_eigenvectorCentrality'].append(eigenvector_centrality[index])
                    features['Nw_averageNeighborDegree0'].append(average_neighbor_degree[source_first])
                    features['Nw_degreeCentrality0'].append(degree_centrality[source_first])
                    features['Nw_inDegreeCentrality0'].append(in_degree_centrality[source_first])
                    features['Nw_outDegreeCentrality0'].append(out_degree_centrality[source_first])
                    features['Nw_eigenvectorCentrality0'].append(eigenvector_centrality[source_first])
                    features['Nw_averageNeighborDegree-1'].append(average_neighbor_degree[last_source_index])
                    features['Nw_degreeCentrality-1'].append(degree_centrality[last_source_index])
                    features['Nw_inDegreeCentrality-1'].append(in_degree_centrality[last_source_index])
                    features['Nw_outDegreeCentrality-1'].append(out_degree_centrality[last_source_index])
                    features['Nw_eigenvectorCentrality-1'].append(eigenvector_centrality[last_source_index])
                    # Stat: Statistical
                    features['Stat_average_kOut'].append(round(mean(degreeList), 1))
                    features['Stat_average_t'].append(round(mean(timeList), 1))
                    features['Stat_average_deltaDays'].append(sources_dataframe.user_created_days.mean())
                    features['Stat_average_statusesCount'].append(sources_dataframe.user_statuses_count.mean())
                    features['Stat_average_followersCount'].append(sources_dataframe.followers_count.mean())
                    features['Stat_average_favouritesCount'].append(sources_dataframe.user_favourites_count.mean())
                    features['Stat_average_friendsCount'].append(sources_dataframe.friends_count.mean())
                    features['Stat_average_listedCount'].append(sources_dataframe.user_listed_count.mean())
                    features['Stat_average_normalizedUserStatusesCount'].append(sources_dataframe.normalized_user_statuses_count.mean())
                    features['Stat_average_normalizedUserFollowersCount'].append(sources_dataframe.normalized_user_followers_count.mean())
                    features['Stat_average_normalizedUserFavouritesCount'].append(sources_dataframe.normalized_user_favourites_count.mean())
                    features['Stat_average_normalizedUserListedCount'].append(sources_dataframe.normalized_user_listed_count.mean())
                    features['Stat_average_normalizedUserFriendsCount'].append(sources_dataframe.normalized_user_friends_count.mean())
                    features['Stat_max_kOut'].append(max(degreeList))
                    features['Stat_min_kOut'].append(min(degreeList))
                processed_dataframe = pd.DataFrame(features)
            pbar.update(1)
    return processed_dataframe

In [19]:
number_of_processes = multiprocessing.cpu_count()

print('Will start {} processes'.format(number_of_processes))
for interval in intervals:
    with Pool(number_of_processes) as pool:
        parameters = []
        number_of_users = len(unique_users.index)
        task_size = math.ceil(number_of_users/number_of_processes)
        for i in range(number_of_processes):
            start_index = i * task_size
            end_index = min((i + 1) * task_size, number_of_users)
            parameters.append((interval, start_index, end_index))
        dataframe_results = pool.starmap(process_data, parameters)
    result = pd.DataFrame()
    result = result.append(dataframe_results)
    config.dump_ml_data(result, interval)
    print('extracted {} of rows'.format(len(result.index)))
    display(result)

Will start 4 processes


  2%|▏         | 20/1253 [00:08<11:43,  1.75it/s]
Process ForkPoolWorker-7:

Process ForkPoolWorker-6:
Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()

  File "/Applications/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Applications/anaconda3/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/Applications/anaconda3/lib/python3.6/multiprocessing/pool.py", line 47, in starmapstar
    return list(itertools.starmap(args[0], args[1]))
  File "<ipython-input-18-16ff479cf92a>", line 91, in process_data
    sources = [x for x in source_candidates if unique_users.iloc[x].time_lapsed <= current_time]
Process ForkPoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.6/multiprocessing/proces

KeyboardInterrupt: 

In [ ]:
process_data